# Test Multimodal Geospatial Images

Use Geemap to get a dataset, and test using HF to try running the data through multimodal vision langauge models to see what they produce. 

In [3]:
import ee
import geemap
import requests
from PIL import Image
from io import BytesIO
from transformers import FlavaProcessor, FlavaModel


In [4]:
# Authenticate and initialize GEE
ee.Authenticate()
ee.Initialize()

In [6]:
# Define the area of interest with coordinates
coordinates = [[-99.222, 46.781], [-93.581, 46.781], [-93.581, 43.500], [-99.222, 43.500]]
aoi = ee.Geometry.Polygon(coordinates)

# Define the dataset and time range
dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
            .filterBounds(aoi) \
            .filterDate('2020-01-01', '2020-12-31') \
            .first()  # Just take the first image as an example

# Get the URL of the image
url = dataset.getDownloadUrl({
    'scale': 30,
    'crs': 'EPSG:4326',
    'region': aoi
})

# Download the image
response = requests.get(url)

EEException: Total request size (18801853136 bytes) must be less than or equal to 50331648 bytes.

In [ ]:

image = Image.open(BytesIO(response.content))

model = FlavaModel.from_pretrained("facebook/flava-full")
processor = FlavaProcessor.from_pretrained("facebook/flava-full")
processed = processor(images=image, return_tensors="pt")

# Load the FLAVA model
model = FlavaForImageTextRetrieval.from_pretrained("facebook/flava-full")

# Pass the preprocessed image to the model
outputs = model(**processed)

# Postprocess the outputs
# For demonstration, let's assume we want to extract the text embeddings
text_embeddings = outputs.text_embeds

# Print the embeddings (or you can perform further analysis)
print(text_embeddings)